# Lista de Tickers das Ações+FIIs+BDRs BR ativos

<b>OBJETIVO: criar uma lista com os códigos (tickers) dos ativos financeiros negociados na Bolsa Brasileira (B3) e que estejam em atividade na data informada pelo usuário(a).

<b><i>*Utiliza a Base de Dados da Investing.com - Financial Markets Worldwide.</i></b> https://www.investing.com/

<b><i>*NÃO contempla ETFs.</i></b>

<b><u>DADOS DE ENTRADA (INPUTS) - (PREENCHIMENTO DO USUÁRIO(A))

In [1]:
#Informar a data desejada
Data_Inicio = "2022-06-27"
Data_Fim = "2022-06-28"

In [2]:
#Informar o caminho (local) onde deseja salvar o arquivo final (Tabela Final)
Salvar_em = r"C:\Users\momil\OneDrive\Gestão - Marcelo Milani\Projetos Hobbies\3. Mercado Financeiro\202206_Tickers Ações+FIIs+BDRs BR ativos"

<b><u>BIBLIOTECAS NECESSÁRIAS

In [3]:
#Biblioteca para análise e manipulação de dados
import pandas as pd
#Biblioteca para coletar dados do mercado financeiro a partir do site "Investing.com"
import investpy
#Biblioteca para acesso a informações de ativos financeiros do Yahoo Finance
import yfinance as yf

<b><u>IDENTIFICAR TODOS OS ATIVOS NA BASE DE DADOS

In [4]:
#Criar um DataFrme com os ativos brasileiros encontrados na base de dados
df_StocksBR = pd.DataFrame(investpy.stocks.get_stocks(country="Brazil"))
#Exibir mensagem
print(f"Total de {len(df_StocksBR)} ativos financeiros encontrados (Ações+FIIs+BDRs). Abaixo estão os 5 primeiros:")
#Exibir 5 primeiras linhas
df_StocksBR.head()

Total de 749 ativos financeiros encontrados (Ações+FIIs+BDRs). Abaixo estão os 5 primeiros:


,country,name,full_name,isin,currency,symbol
0,brazil,ABC BRASIL PN,Banco ABC Brasil SA,BRABCBACNPR4,BRL,ABCB4
1,brazil,BRASILAGRO ON,BrasilAgro - Co ON NM,BRAGROACNOR7,BRL,AGRO3
2,brazil,RUMO ON NM,RUMO Logistica Operadora Multimodal SA,BRRAILACNOR9,BRL,RAIL3
3,brazil,ALPARGATAS ON,Alpargatas SA,BRALPAACNOR0,BRL,ALPA3
4,brazil,ALPARGATAS PN,Alpargatas SA,BRALPAACNPR7,BRL,ALPA4


<b><u>CRIAR VETOR COM OS CÓDIGOS DOS ATIVOS (TICKERS)

In [5]:
#Adicionar o texto ".SA" ao final do ticker -> Necessário para coletar dados no Yahoo Finance
df_StocksBR["Codigo_Ativo"] = df_StocksBR["symbol"] + ".SA"

In [6]:
#Vetor com os códigos dos ativos
array_codigo_ativos = df_StocksBR["Codigo_Ativo"].values

<b><u>COLETAR PREÇOS DOS ATIVOS NA DATA INFORMADA PELO USUÁRIO(A)

In [7]:
#DataFrame para armazenar preço dos ativos
Precos = pd.DataFrame()
#Coletar ativo por ativo
for ativo in array_codigo_ativos:
    Precos[ativo] = yf.download(ativo, start=Data_Inicio, end=Data_Fim, progress=False, show_errors=False)["Adj Close"]

C:\Users\momil\AppData\Local\Temp/ipykernel_19300/3604605315.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  Precos[ativo] = yf.download(ativo, start=Data_Inicio, end=Data_Fim, progress=False, show_errors=False)["Adj Close"]


<b><u>ELIMINAR ATIVOS QUE NÃO POSSUEM COTAÇÃO NA DATA INFORMADA PELO USUÁRIO(A)

In [8]:
#Ajustar DataFrame (Linha para Coluna)
Precos = Precos.transpose().reset_index()

In [9]:
#Renomear coluna com os códigos dos ativos
Precos.rename(columns={"index":"Codigo_Ativo"}, inplace=True)

In [10]:
#Eliminar ativos sem cotação na data informada
Precos.dropna(inplace=True)

<b><u>CRIAR TABELA FINAL COM OS ATIVOS

In [11]:
#Eliminar o ".SA" dos códigos dos ativos
Precos["Codigo_Ativo"] = Precos["Codigo_Ativo"].apply(lambda x: x[:-3])
#Eliminar possíveis espaços nas laterais
Precos["Codigo_Ativo"] = Precos["Codigo_Ativo"].str.strip()

In [12]:
#Criar DataFrame com as informações
df_StocksBR_Final = pd.merge(Precos["Codigo_Ativo"], df_StocksBR[["symbol","name","full_name","country"]],
                             how="left", left_on="Codigo_Ativo", right_on="symbol")

In [13]:
#Retirar possíveis duplicados
df_StocksBR_Final = df_StocksBR_Final.drop_duplicates(subset=["Codigo_Ativo"], keep="first")
#Excluir coluna "symbol"
df_StocksBR_Final.drop(columns="symbol", errors="ignore", inplace=True)
#Renomear colunas
df_StocksBR_Final.rename(columns={"name":"Nome_Ativo", "full_name":"Nome_Completo_Ativo", "country":"País"}, inplace=True)

In [14]:
#Exibir mensagem
print(f"Total de {len(df_StocksBR_Final)} ativos financeiros (Ações+FIIs+BDRs). Abaixo estão os 5 primeiros:")
#Exibir 5 primeiras linhas
df_StocksBR_Final.head()

Total de 594 ativos financeiros (Ações+FIIs+BDRs). Abaixo estão os 5 primeiros:


,Codigo_Ativo,Nome_Ativo,Nome_Completo_Ativo,País
0,ABCB4,ABC BRASIL PN,Banco ABC Brasil SA,brazil
1,AGRO3,BRASILAGRO ON,BrasilAgro - Co ON NM,brazil
2,RAIL3,RUMO ON NM,RUMO Logistica Operadora Multimodal SA,brazil
3,ALPA3,ALPARGATAS ON,Alpargatas SA,brazil
4,ALPA4,ALPARGATAS PN,Alpargatas SA,brazil


<b><u>GERAR E EXPORTAR EXCEL

In [15]:
df_StocksBR_Final.to_excel(rf"{Salvar_em}\Lista de Ativos_Acoes_FII_BDR.xlsx", index=False)